In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/wns-hack-v1/__output__.json
/kaggle/input/wns-hack-v1/trainWithLogMergeApprox.csv
/kaggle/input/wns-hack-v1/testWithLogMergeApprox.csv
/kaggle/input/wns-hack-v1/__notebook__.ipynb
/kaggle/input/wns-hack-v1/__results__.html
/kaggle/input/wns-hack-v1/custom.css


In [2]:
train = pd.read_csv('../input/wns-hack-v1/trainWithLogMergeApprox.csv')
test = pd.read_csv('../input/wns-hack-v1/testWithLogMergeApprox.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26,28,30,32,34,36,38,40,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
###
target = train['is_click']
c = [i for i in train.columns if i in test.columns]
train = train[c]
test = test[c]

In [4]:
train.fillna(-99,inplace=True)
test.fillna(-99,inplace=True)

In [5]:
badIdxTr = train[train['NearestLog_server_time']=='0']['NearestLog_server_time'].index
badIdxTe = test[test['NearestLog_server_time']=='0']['NearestLog_server_time'].index
train.loc[badIdxTr,'NearestLog_server_time'] = '2018-01-01 00:00:00'
test.loc[badIdxTe,'NearestLog_server_time'] = '2018-01-01 00:00:00'

In [6]:
def getTimeDelta(train,test):
    trTD = (pd.to_datetime(train['impression_time']) -pd.to_datetime(train['NearestLog_server_time']))
    teTD = (pd.to_datetime(test['impression_time']) -pd.to_datetime(test['NearestLog_server_time']))
    train['TD_Days']=trTD.dt.days
    train['TD_Hours']=(trTD.dt.seconds)//3600
    train['TD_Mins']=(trTD.dt.seconds//60)%60
    test['TD_Days']=teTD.dt.days
    test['TD_Hours']=(teTD.dt.seconds)//3600
    test['TD_Mins']=(teTD.dt.seconds//60)%60
    return train,test

train,test = getTimeDelta(train,test)

In [7]:
def converttoDT(train,test):
    train['impression_time'] = pd.to_datetime(train['impression_time'])
    test['impression_time'] = pd.to_datetime(test['impression_time'])
    test['NearestLog_server_time'] = pd.to_datetime(test['NearestLog_server_time'])
    train['NearestLog_server_time'] = pd.to_datetime(train['NearestLog_server_time'])
    return train,test

train,test = converttoDT(train,test)

In [8]:
pd.set_option('display.max_columns',500)
test.head(1)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,impression_time_day,impression_time_minute,impression_time_second,LogPresent_24_hour_before,Count_LogPresent_24_hour_before,LogPresent_192_hour_before,Count_LogPresent_192_hour_before,LogPresent_288_hour_before,Count_LogPresent_288_hour_before,NearestLog_server_time,NearestLog_user_id,NearestLog_device_type_0,NearestLog_item_id_0,NearestLog_device_type_1,NearestLog_item_id_1,NearestLog_device_type_2,NearestLog_item_id_2,NearestLog_device_type_3,NearestLog_item_id_3,NearestLog_device_type_4,NearestLog_item_id_4,NearestLog_device_type_5,NearestLog_item_id_5,NearestLog_device_type_6,NearestLog_item_id_6,NearestLog_device_type_7,NearestLog_item_id_7,TD_Days,TD_Hours,TD_Mins
0,a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,latest,1,13,44,12,0,0,1,1,1,1,2018-12-11 19:01:00,44754,android,128995.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,1,12,43


In [9]:
def getCountImpressionPrevDHour(minsArr,D):
    casdt = list()
    casdt.append(0)
    for i in range(len(minsArr)):
        if i == 0:
            pass
        elif i == 1:
            casdt.append(((minsArr[1]-minsArr[0])<D).astype(int))
        elif i > 1:
            i += 1
            newArr = minsArr[:i]
            cumsum = np.cumsum(newArr)
            res = cumsum[-1]-cumsum[:-1]
            res = np.sum(res<D)
            casdt.append(res)
    return pd.Series(casdt)

In [10]:
def getImpressionCounts(df,col):
    df[col+'_timeSinceLastImpressionMins'] = 0
    df[col+'_AvgtimeForLastImpressionMins'] = 0
    df[col+'_StdtimeForLastImpressionMins'] = 0
    df[col+'_NCountImpressions'] = 0
    for uq in tqdm(df[col].unique()):
        temp = df[df[col]==uq]['impression_time'].sort_values()
        shift = temp.shift(1)
        diff = (temp.values-temp.shift(1,fill_value=temp.values.min()).values)
        minsArr = diff.astype('timedelta64[m]') / np.timedelta64(1, 'm')
        df.loc[temp.index,col+'_timeSinceLastImpressionMins'] = np.array(minsArr)
        df.loc[temp.index,col+'_AvgtimeForLastImpressionMins'] = np.mean(minsArr)
        df.loc[temp.index,col+'_StdtimeForLastImpressionMins'] = np.std(minsArr)
        df.loc[temp.index,col+'_NCountImpressions'] = len(temp)
    return df

In [11]:
from tqdm import tqdm_notebook
def getCountinLastDMins(df,col,D=[60,120]):
    for d in D:
        df['{}_CountInLast_{}_MinsImpression'.format(col,d)] = 0
    for uq in tqdm(df[col].unique()):
        temp = df[df[col]==uq]['impression_time'].sort_values()
        shift = temp.shift(1)
        diff = (temp.values-temp.shift(1,fill_value=temp.values.min()).values)
        minsArr = diff.astype('timedelta64[m]') / np.timedelta64(1, 'm')
        for d in D:
            df.loc[temp.index,'{}_CountInLast_{}_MinsImpression'.format(col,d)] = getCountImpressionPrevDHour(minsArr,d).values
    return df

In [12]:
consol = pd.concat([train,test],0)
consol.reset_index(inplace=True,drop=True)
consol.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,impression_time_day,impression_time_minute,impression_time_second,LogPresent_24_hour_before,Count_LogPresent_24_hour_before,LogPresent_192_hour_before,Count_LogPresent_192_hour_before,LogPresent_288_hour_before,Count_LogPresent_288_hour_before,NearestLog_server_time,NearestLog_user_id,NearestLog_device_type_0,NearestLog_item_id_0,NearestLog_device_type_1,NearestLog_item_id_1,NearestLog_device_type_2,NearestLog_item_id_2,NearestLog_device_type_3,NearestLog_item_id_3,NearestLog_device_type_4,NearestLog_item_id_4,NearestLog_device_type_5,NearestLog_item_id_5,NearestLog_device_type_6,NearestLog_item_id_6,NearestLog_device_type_7,NearestLog_item_id_7,TD_Days,TD_Hours,TD_Mins
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,15,0,11,0,0,0,0,0,0,2018-10-19 18:11:00,87862,android,43886.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,26,5,49
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,15,1,11,0,0,0,0,0,0,2018-10-25 05:03:00,63410,android,43209.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,20,18,58
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,15,2,11,0,0,0,0,0,0,2018-10-23 12:59:00,71748,android,122348.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,22,11,3
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,15,2,11,1,8,1,11,1,11,2018-11-14 19:01:00,69209,android,1528.0,android,2715.0,android,104035.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,0,5,1
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,15,2,11,1,2,1,7,1,7,2018-11-14 21:42:00,62873,android,4069.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,-99,-99.0,0,2,20


In [13]:
consol.index.nunique() == consol.shape[0]

True

In [14]:
from tqdm import tqdm
D=[60,120,300,600,24*60,24*60*2,24*60*5,24*60*7,24*60*10,24*60*14,24*60*17,24*60*21,24*60*24,24*60*28,24*60*35,24*60*42,24*60*49]
for c in tqdm(['user_id','app_code','NearestLog_item_id_0']):
    consol = getImpressionCounts(consol,c)
    consol = getCountinLastDMins(consol,c,D=D)
    print(consol.tail())

 14%|█▍        | 12404/89157 [04:37<28:05, 45.53it/s]

In [15]:
train = consol.iloc[:train.shape[0]]
test = consol.iloc[train.shape[0]:]
train['is_click'] = target

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
train.to_csv('trainLogMergeImpressionTimeStats.csv',index=False)
test.to_csv('testLogMergeImpressionTimeStats.csv',index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [17]:
train[train['NearestLog_item_id_0_CountInLast_2880_MinsImpression']>0]['NearestLog_item_id_0_CountInLast_2880_MinsImpression'].unique()

array([  1,   2,   5,   4,   3,  11,   6,   7,  17,   8,  18,  20,  21,
        24,  25,  27,   9,  10,  33,  29,  13,  12,  40,  14,  15,  52,
        53,  19,  54,  16,  61,  71,  23,  22,  77,  26,  80,  89,  32,
        34,  98,  30,  35,  36,  41,  39,  38,  37,  48,  46,  60, 108,
        31,  42,  43, 120,  45,  62,  69,  66,  44,  47, 122,  28,  50,
        51, 128, 132,  70,  72,  74,  82,  58, 134, 133,  87,  88,  92,
        63, 135, 139, 140, 103, 145, 148, 101, 107,  79,  81,  56, 109,
        83, 165, 167, 112, 114,  85,  90,  99,  65, 173, 164, 100, 169,
       127, 124, 125, 126, 129, 106, 111,  75,  67, 184,  55,  49,  57,
       188, 176,  64,  68, 115, 121, 181, 200, 185,  96,  91, 156, 160,
       174, 177, 161,  59, 159, 171, 162, 170, 178,  76, 206, 208,  84,
       209, 212, 211,  86, 213, 217, 219, 221, 113, 110, 189,  93, 225,
        78, 230, 116, 119,  73, 197, 102, 137, 258, 244, 248, 261, 192,
       194, 195, 252, 264, 265, 262, 187, 203, 155, 198, 271, 27

In [18]:
pd.set_option('display.max_rows',500)
train.isnull().sum()/train.shape[0]

impression_id                                            0.0
impression_time                                          0.0
user_id                                                  0.0
app_code                                                 0.0
os_version                                               0.0
is_4G                                                    0.0
impression_time_day                                      0.0
impression_time_minute                                   0.0
impression_time_second                                   0.0
LogPresent_24_hour_before                                0.0
Count_LogPresent_24_hour_before                          0.0
LogPresent_192_hour_before                               0.0
Count_LogPresent_192_hour_before                         0.0
LogPresent_288_hour_before                               0.0
Count_LogPresent_288_hour_before                         0.0
NearestLog_server_time                                   0.0
NearestLog_user_id      